In [1]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.graphics.regressionplots import influence_plot
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
df=pd.read_csv("F:/n/IMDb Movies India.csv",encoding='latin1')
df

,Name,Year,Duration,Genre,Rating,Votes,Director,Actor 1,Actor 2,Actor 3
0,,NaN,NaN,Drama,NaN,NaN,J.S. Randhawa,Manmauji,Birbal,Rajendra Bhatia
1,#Gadhvi (He thought he was Gandhi),-2019.0,109 min,Drama,7.0,8,Gaurav Bakshi,Rasika Dugal,Vivek Ghamande,Arvind Jangid
2,#Homecoming,-2021.0,90 min,"Drama, Musical",NaN,NaN,Soumyajit Majumdar,Sayani Gupta,Plabita Borthakur,Roy Angana
3,#Yaaram,-2019.0,110 min,"Comedy, Romance",4.4,35,Ovais Khan,Prateik,Ishita Raj,Siddhant Kapoor
4,...And Once Again,-2010.0,105 min,Drama,NaN,NaN,Amol Palekar,Rajat Kapoor,Rituparna Sengupta,Antara Mali
...,...,...,...,...,...,...,...,...,...,...
15504,Zulm Ko Jala Doonga,-1988.0,NaN,Action,4.6,11,Mahendra Shah,Naseeruddin Shah,Sumeet Saigal,Suparna Anand
15505,Zulmi,-1999.0,129 min,"Action, Drama",4.5,655,Kuku Kohli,Akshay Kumar,Twinkle Khanna,Aruna Irani
15506,Zulmi Raj,-2005.0,NaN,Action,NaN,NaN,Kiran Thej,Sangeeta Tiwari,NaN,NaN
15507,Zulmi Shikari,-1988.0,NaN,Action,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15509 entries, 0 to 15508
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Name      15509 non-null  object 
 1   Year      14981 non-null  float64
 2   Duration  7240 non-null   object 
 3   Genre     13632 non-null  object 
 4   Rating    7919 non-null   float64
 5   Votes     7920 non-null   object 
 6   Director  14984 non-null  object 
 7   Actor 1   13892 non-null  object 
 8   Actor 2   13125 non-null  object 
 9   Actor 3   12365 non-null  object 
dtypes: float64(2), object(8)
memory usage: 1.2+ MB


In [4]:
df.isnull().sum()

Name           0
Year         528
Duration    8269
Genre       1877
Rating      7590
Votes       7589
Director     525
Actor 1     1617
Actor 2     2384
Actor 3     3144
dtype: int64

In [5]:

df = df.dropna(subset=['Rating'])

# Convert 'Votes' to numeric, coerce errors to NaN
df['Votes'] = pd.to_numeric(df['Votes'], errors='coerce')

# Fill missing values in 'Votes' with the median value
df['Votes'] = df['Votes'].fillna(df['Votes'].median())

# Extract numeric values from 'Duration'
df['Duration'] = df['Duration'].str.extract('(\d+)').astype(float)

# Fill missing values in 'Duration' with the median value
df['Duration'] = df['Duration'].fillna(df['Duration'].median())

# Combine all actor columns into a single column for simplicity
df['Actors'] = df[['Actor 1', 'Actor 2', 'Actor 3']].apply(lambda x: ', '.join(x.dropna()), axis=1)

# Drop the individual actor columns
df = df.drop(columns=['Actor 1', 'Actor 2', 'Actor 3'])
df

C:\Users\chait\AppData\Local\Temp\ipykernel_11460\1493703859.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Votes'] = pd.to_numeric(df['Votes'], errors='coerce')
C:\Users\chait\AppData\Local\Temp\ipykernel_11460\1493703859.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Votes'] = df['Votes'].fillna(df['Votes'].median())
C:\Users\chait\AppData\Local\Temp\ipykernel_11460\1493703859.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

,Name,Year,Duration,Genre,Rating,Votes,Director,Actors
1,#Gadhvi (He thought he was Gandhi),-2019.0,109.0,Drama,7.0,8.0,Gaurav Bakshi,"Rasika Dugal, Vivek Ghamande, Arvind Jangid"
3,#Yaaram,-2019.0,110.0,"Comedy, Romance",4.4,35.0,Ovais Khan,"Prateik, Ishita Raj, Siddhant Kapoor"
5,...Aur Pyaar Ho Gaya,-1997.0,147.0,"Comedy, Drama, Musical",4.7,827.0,Rahul Rawail,"Bobby Deol, Aishwarya Rai Bachchan, Shammi Kapoor"
6,...Yahaan,-2005.0,142.0,"Drama, Romance, War",7.4,35.0,Shoojit Sircar,"Jimmy Sheirgill, Minissha Lamba, Yashpal Sharma"
8,?: A Question Mark,-2012.0,82.0,"Horror, Mystery, Thriller",5.6,326.0,Allyson Patel,"Yash Dave, Muntazir Ahmad, Kiran Bhatia"
...,...,...,...,...,...,...,...,...
15501,Zulm Ki Hukumat,-1992.0,134.0,"Action, Crime, Drama",5.3,135.0,Bharat Rangachary,"Dharmendra, Moushumi Chatterjee, Govinda"
15503,Zulm Ki Zanjeer,-1989.0,125.0,"Action, Crime, Drama",5.8,44.0,S.P. Muthuraman,"Chiranjeevi, Jayamalini, Rajinikanth"
15504,Zulm Ko Jala Doonga,-1988.0,134.0,Action,4.6,11.0,Mahendra Shah,"Naseeruddin Shah, Sumeet Saigal, Suparna Anand"
15505,Zulmi,-1999.0,129.0,"Action, Drama",4.5,655.0,Kuku Kohli,"Akshay Kumar, Twinkle Khanna, Aruna Irani"


In [6]:
X = df[['Year', 'Duration', 'Votes']]
y = df['Rating']


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=40)

# Train a random forest regressor
rf = RandomForestRegressor(n_estimators=100, random_state=40)
rf.fit(X_train, y_train)

# Evaluate the model
y_pred = rf.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'MSE: {mse:.2f}')

MSE: 1.69


In [7]:
df2 = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df2

,Actual,Predicted
11335,5.7,5.908000
12091,6.3,5.497000
12256,5.8,6.693000
4365,7.0,6.388000
652,4.7,5.457750
...,...,...
10451,6.7,6.346333
2612,6.0,5.193000
272,3.2,4.903950
10467,7.0,7.439000
